In [0]:
from bs4 import BeautifulSoup
import requests
import sqlite3
import pandas as pd


TIKI_URL = 'https://tiki.vn'

In [0]:
conn = sqlite3.connect('/content/drive/My Drive/TongaPTA/Tiki-Prod/Tiki-prod.db')
cur = conn.cursor()

In [0]:
def create_categories_table():
    query = """
        CREATE TABLE IF NOT EXISTS categories (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name VARCHAR(255),
            url TEXT, 
            parent_id INT, 
            create_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    """
    try:
        cur.execute(query)
    except Exception as err:
        print('ERROR BY CREATE TABLE', err)
create_categories_table()

In [0]:
class Category:
    def __init__(self, cat_id, name, url, parent_id):
        self.cat_id = cat_id
        self.name = name
        self.url = url
        self.parent_id = parent_id

    def __repr__(self):
        return "ID: {}, Name: {}, URL: {}, Parent_id: {}".format(self.cat_id, self.name, self.url, self.parent_id)

    def save_into_db(self):
        query = """
            INSERT INTO categories (name, url, parent_id)
            VALUES (?, ?, ?);
        """
        val = (self.name, self.url, self.parent_id)
        try:
            cur.execute(query, val)
            self.cat_id = cur.lastrowid
        except Exception as err:
            print('ERROR BY INSERT:', err)

In [0]:
def get_url(url):
    # time.sleep(1)
    try:
        response = requests.get(url).text
        response = BeautifulSoup(response, 'html.parser')
        return response
    except Exception as err:
            print('ERROR BY REQUEST:', err)

In [0]:
def get_main_categories(save_db=False):
    soup = get_url(TIKI_URL)

    result = []
    for a in soup.findAll('a', {'class':'MenuItem__MenuLink-tii3xq-1 efuIbv'}):
        cat_id = None
        name = a.find('span', {'class':'text'}).text
        url = a['href']
        parent_id = None

        cat = Category(cat_id, name, url, parent_id)
        if save_db:
            cat.save_into_db()
        result.append(cat)
    return result

In [0]:
main_categories = get_main_categories(save_db=True)

In [0]:
def get_sub_categories(category, save_db=False):
    name = category.name
    url = category.url
    result = []

    try:
        soup = get_url(url)
        div_containers = soup.findAll('div', {'class':'list-group-item is-child'})
        for div in div_containers:
            sub_id = None
            sub_name = div.a.text
            sub_url = 'http://tiki.vn' + div.a['href']
            sub_parent_id = category.cat_id

            sub = Category(sub_id, sub_name, sub_url, sub_parent_id)
            if save_db:
                sub.save_into_db()
            result.append(sub)
    except Exception as err:
        print('ERROR BY GET SUB CATEGORIES:', err)

    return result

In [0]:
from collections import deque

de = deque([1, 2, 3, 4, 5,6])

In [0]:
def get_all_categories(main_categories):
    de = deque(main_categories)
    count = 0

    while de:
        parent_cat = de.popleft()
        sub_cats = get_sub_categories(parent_cat, save_db=True)
        # print(sub_cats)
        de.extend(sub_cats)
        count += 1

        if count % 100 == 0:
            print(count, 'times')


In [12]:
get_all_categories(main_categories)

100 times
200 times
300 times
400 times
500 times
600 times
700 times
800 times
900 times
1000 times
1100 times
1200 times
1300 times
1400 times
1500 times
1600 times
1700 times
1800 times
1900 times
2000 times
2100 times
2200 times
2300 times
2400 times
2500 times
2600 times
2700 times
2800 times
2900 times
3000 times
3100 times


In [0]:
conn.commit()

In [15]:
pd.read_sql_query('''SELECT *
                    FROM categories ''', conn)

,id,name,url,parent_id,create_at
0,1,Điện Thoại - Máy Tính Bảng,https://tiki.vn/dien-thoai-may-tinh-bang/c1789...,NaN,2020-03-15 22:46:38
1,2,Điện Tử - Điện Lạnh,https://tiki.vn/tivi-thiet-bi-nghe-nhin/c4221?...,NaN,2020-03-15 22:46:38
2,3,Phụ Kiện - Thiết Bị Số,https://tiki.vn/thiet-bi-kts-phu-kien-so/c1815...,NaN,2020-03-15 22:46:38
3,4,Laptop - Thiết bị IT,https://tiki.vn/laptop-may-vi-tinh/c1846?src=c...,NaN,2020-03-15 22:46:38
4,5,Máy Ảnh - Quay Phim,https://tiki.vn/may-anh/c1801?src=c.1801.hambu...,NaN,2020-03-15 22:46:38
...,...,...,...,...,...
3193,3194,\n Viết ...,http://tiki.vn/viet/c892?src=c.8322.hamburger_...,3082.0,2020-03-16 00:07:03
3194,3195,\n Đọc ...,http://tiki.vn/doc/c889?src=c.8322.hamburger_m...,3082.0,2020-03-16 00:07:03
3195,3196,\n IELTS ...,http://tiki.vn/ielts/c895?src=c.8322.hamburger...,3083.0,2020-03-16 00:07:05
3196,3197,\n TOEFL ...,http://tiki.vn/toeflt/c894?src=c.8322.hamburge...,3083.0,2020-03-16 00:07:05


In [29]:
pd.read_sql_query('''SELECT a.id, a.name, a.url, a.parent_id, a.create_at
                    FROM categories AS a 
                    LEFT JOIN categories AS b ON a.id = b.parent_id
                    WHERE b.parent_id is null   
                    LIMIT 10 OFFSET 100''', conn)

,id,name,url,parent_id,create_at
0,285,\n Máy chấm côn...,http://tiki.vn/may-cham-cong/c12886?src=c.1846...,44,2020-03-15 22:47:59
1,286,\n Máy hủy tài ...,http://tiki.vn/may-huy-tai-lieu/c10089?src=c.1...,44,2020-03-15 22:47:59
2,287,\n Máy scan mã ...,http://tiki.vn/may-scan-ma-vach/c30020?src=c.1...,44,2020-03-15 22:47:59
3,288,\n Máy đếm tiền...,http://tiki.vn/may-dem-tien/c29006?src=c.1846....,44,2020-03-15 22:47:59
4,290,\n Phần mềm máy...,http://tiki.vn/phan-mem-may-tinh/c6255?src=c.1...,44,2020-03-15 22:47:59
5,291,\n Thiết bị tru...,http://tiki.vn/thiet-bi-truyen-thong-hoi-nghi/...,44,2020-03-15 22:47:59
6,292,\n Bao đựng chố...,http://tiki.vn/bao-dung-chong-sock-chong-nuoc/...,45,2020-03-15 22:48:00
7,293,\n Dây đeo máy ...,http://tiki.vn/day-deo-may-anh-may-quay-ong-nh...,45,2020-03-15 22:48:00
8,294,"\n Hộp Đựng, Tủ...",http://tiki.vn/hop-dung-tu-chong-am/c1843?src=...,45,2020-03-15 22:48:00
9,295,\n Hộp đựng Fil...,http://tiki.vn/hop-dung-filter/c28786?src=c.18...,45,2020-03-15 22:48:00


#Craw product from Tiki's subcategories:

In [0]:
#Create product table
def create_product_table():
    query = """
        CREATE TABLE IF NOT EXISTS products (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            cat_id INT,
            product_id INT,
            name VARCHAR(255),
            tiki_now VARCHAR(255),
            normal_price VARCHAR(255),
            final_price VARCHAR(255),
            img_url TEXT, 
            ref_url TEXT,
            create_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    """
    try:
        cur.execute(query)
    except Exception as err:
        print('ERROR BY CREATE TABLE', err)
create_product_table()

In [18]:
pd.read_sql_query('''SELECT *
                    FROM products ''', conn)

,id,cat_id,product_id,name,tiki_now,normal_price,final_price,img_url,ref_url,create_at


In [0]:
# Create Product Class
class Product:
    def __init__(self, cat_id, product_id, name, tiki_now, normal_price, final_price, img_url, ref_url):
        self.cat_id = cat_id
        self.product_id = product_id
        self.name = name
        self.tiki_now = tiki_now
        self.normal_price = normal_price
        self.final_price = final_price
        self.img_url = img_url
        self.ref_url = ref_url
    
    def __repr__(self):
        return """CatID: {},ID: {}, Name: {}, tiki_now: {} , Normal_price: {} , Final_price: {}, 
                IMG_URL: {}, IMG_URL: {}""".format(self.cat_id, self.product_id, self.name, self.tiki_now, self.normal_price,self.final_price ,self.img_url , self.ref_url)

    def save_into_db(self):
        query = """
            INSERT INTO products (cat_id, product_id, name, tiki_now, normal_price,final_price ,img_url , ref_url)
            VALUES (?, ?, ?, ? ,?, ? ,?, ?);
        """
        val = (self.cat_id, self.product_id, self.name, self.tiki_now, self.normal_price,self.final_price ,self.img_url , self.ref_url)
        try:
            cur.execute(query, val)
        except Exception as err:
            print('ERROR BY INSERT:', err)

In [0]:
def get_product(key,URL,save_db=False):   
    soup = get_url(URL)
    items = soup.find_all('div',class_='product-item')
    result = []
    
    for item in items:
        try:
            cat_id = key
            product_id = item['data-id']
            name = item.a['title']
            normal_price = item.find('span',class_='price-regular').string
            final_price = item.find('span',class_='final-price').contents[0].strip() 
            img_url = item.img['src']
            ref_url = item.a['href']
            if  item.i['class'][0]:
                tiki_now = item.i['class'][0]
            
            item = Product(cat_id, product_id, name, tiki_now, normal_price, final_price, img_url, ref_url)
            if save_db:
               item.save_into_db()
            result.append(item)
        except Exception as err:
            print('ERROR BY GET PRODUCT:', err)
    return result

In [0]:
# only run crawing data on smallest level of sub
links = dict(cur.execute("""SELECT a.id, a.url
                            FROM categories AS a 
                            LEFT JOIN categories AS b ON a.id = b.parent_id
                            WHERE b.parent_id is null""").fetchall())

In [57]:
for key in links.keys():
    get_product(key,links[key],save_db=True)


Streaming output truncated to the last 5000 lines.
ERROR BY GET PRODUCT: 'NoneType' object is not subscriptable
ERROR BY GET PRODUCT: 'NoneType' object is not subscriptable
ERROR BY GET PRODUCT: 'NoneType' object is not subscriptable
ERROR BY GET PRODUCT: 'NoneType' object is not subscriptable
ERROR BY GET PRODUCT: 'NoneType' object is not subscriptable
ERROR BY GET PRODUCT: 'NoneType' object is not subscriptable
ERROR BY GET PRODUCT: 'NoneType' object is not subscriptable
ERROR BY GET PRODUCT: 'NoneType' object is not subscriptable
ERROR BY GET PRODUCT: 'NoneType' object is not subscriptable
ERROR BY GET PRODUCT: 'NoneType' object is not subscriptable
ERROR BY GET PRODUCT: 'NoneType' object is not subscriptable
ERROR BY GET PRODUCT: 'NoneType' object is not subscriptable
ERROR BY GET PRODUCT: 'NoneType' object is not subscriptable
ERROR BY GET PRODUCT: 'NoneType' object is not subscriptable
ERROR BY GET PRODUCT: 'NoneType' object is not subscriptable
ERROR BY GET PRODUCT: 'NoneType' o

In [64]:
pd.read_sql_query('''SELECT count(*)
                    FROM products ''', conn)

,count(*)
0,63262


#SIMPLE Analysis

In [65]:
# How many products found
pd.read_sql_query('''SELECT DISTINCT(id)
                    FROM products ''', conn)

,count(*)
0,63262


In [72]:
# Grouping product by tiki_now attribute
pd.read_sql_query('''SELECT count(*),tiki_now
                    FROM products 
                    GROUP BY tiki_now''', conn)

,count(*),tiki_now
0,508,ico
1,13,icon-save-sub
2,26548,star
3,36193,tikicon
